In [ ]:
import json
import pathlib
from dotenv import load_dotenv

import openai
from langchain.llms import OpenAI
from langchain.docstore import Wikipedia
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.agents.react.base import DocstoreExplorer
from langchain.agents import load_tools
from langchain.tools import StructuredTool

import os
from pathlib import Path


from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader
from langchain.embeddings import HuggingFaceBgeEmbeddings

from langchain.chains import RetrievalQAWithSourcesChain


In [ ]:
# openai.api_key = os.getenv("AZURE_OPENAI_KEY")
# openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT") # your endpoint should look like the following https://YOUR_RESOURCE_NAME.openai.azure.com/
# openai.api_type = 'azure'
# openai.api_version = '2023-05-15' # this may change in the future
from langchain.llms import AzureOpenAI

In [ ]:
load_dotenv(dotenv_path=pathlib.Path('')) # ".env"

llm = OpenAI(temperature=0, model_name="gpt-3.5-turbo") #gpt-3.5-turbo-16k # 
#os.environ['OPENAI_API_KEY'] = 
# llm = AzureOpenAI(
#     deployment_name="gpt-35-turbo-16k",
#     model_name="gpt-35-turbo-16k",
# )

## Vector store

In [ ]:
loader = DirectoryLoader("./demo_app", glob="**/*.*", show_progress=True, use_multithreading=True, loader_cls=TextLoader)
docs = loader.load()
print(len(docs))

In [ ]:
model_name = "BAAI/bge-small-en"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [ ]:
from langchain.document_loaders import TextLoader


text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(docs)

embeddings = hf
docsearch = Chroma.from_documents(texts, embeddings, collection_name="state-of-union")

## Agent instantiation

In [ ]:
tools = []

In [ ]:
tools = load_tools(["llm-math"], llm=llm)

In [ ]:

import os
from langchain.tools import tool



class Docstore:
    def __init__(self, folder_path):
        self.folder_path = folder_path

    def list_files(self, *args, **kwargs):
        """List all files in the document store recursively."""
        all_files = []
        for dirpath, dirnames, filenames in os.walk(self.folder_path):
            for filename in filenames:
                all_files.append(os.path.join(dirpath, filename))
        return all_files

    def read_file(self, filepath):
        """Read the content of a specific file."""
        filepath = filepath.replace("'", "")
        with open(filepath, 'rb') as file:
            return file.read()
        



def create_ticket(title: str, description: str, affected_files=[]):
    """Create a ticket or code feature request as an output JSON file. with the given input parameters"""
    output_folder = "./tickets/"
    # Create a ticket dictionary with the provided details
    ticket = {
        "title": title,
        "description": description,
        "affected_files": affected_files,
        "status": "Open"
    }
    
    # Generate a unique filename based on the title
    filename = title.replace(" ", "_") + ".json"
    filepath = os.path.join(output_folder, filename)
    
    # Write the ticket to a JSON file
    with open(filepath, 'w') as file:
        json.dump(ticket, file, indent=4)
    
    return f"Ticket saved to {filepath}"

docstore = Docstore(folder_path="./demo_app/backend/app/app/")

# tools.append(
#     Tool(
#         name="ListFiles",
#         func=docstore.list_files,
#         description="Lists all txt files in the document store.",
#     )
# )

# tools.append(
#     Tool(
#         name="ReadFile",
#         func=docstore.read_file,
#         description="Reads the content of a specific code file.",
#     )
# )

ticket_tool = StructuredTool.from_function(create_ticket)



tools.append(
    ticket_tool
)

In [ ]:
repo = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
)

vector_store_tool = Tool(
    name="Code repository",
    func=repo.__call__,
    description="useful for when you need to answer questions about code files within the repo. Input should be a fully formed question about what code files to search.",
)
tools.append(vector_store_tool)

In [ ]:
tech_lead_agent_system_prompt: str = """
You are a tech lead, you have access to multiple repos, given a feature request, your job is to go through the code files and create a list of tickets for a software engineer to work on

"""

In [ ]:
react = initialize_agent(tools, llm, agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [ ]:
feature_request: str = ""

In [ ]:
request_prompt_template: str = """
Langchain Agent:

Given the repository's vector store containing all source code files, analyze the following user request:

allow user to change password

Your objective is to critically evaluate the repository to determine which files will be affected by the above request. Based on your analysis, create a set of tickets/issues that outline distinct units of work. Each ticket should:

1. Describe the specific work to be done.
2. Detail the changes required.
3. List the relevant files that will be impacted.

Ensure that each ticket is clear and concise, providing enough information for a software engineer to understand and implement the necessary changes.

Once you've created the tickets, save them to the designated ticketing system or location for future reference and action by the engineering team.

"""

In [ ]:
react.run(
    request_prompt_template
)

In [ ]:
query = "poetry"
docs = docsearch.similarity_search(query)
print(docs[2].page_content)

In [ ]:
len(docs)